# Deteccion de Fugados en accidentes

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Hemos detectado que se repetía una serie de datos nulos en varias columnas, siempre coincidiendo

Al momento de observar los nulos en la columna 'tipo_vehiculo' nos dimos cuenta que coincidía que 6 columnas son nulas, nuestra teoría es que son personas fugadas, además solo añade los conductores

In [3]:
combined_data = pd.read_csv('./Datasets/Data_Combinada.csv')
combined_data.filter(items=[ 'sexo','positiva_droga', 'positiva_alcohol', 'tipo_vehiculo', 'tipo_persona', 'rango_edad', 'tipo_accidente'])[combined_data['tipo_persona'] == 'Conductor'][combined_data['tipo_vehiculo'].isnull()][combined_data['sexo'] == 'Desconocido']

C:\Users\Usuario\AppData\Local\Temp\ipykernel_11096\2914296293.py:1: DtypeWarning: Columns (15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_data = pd.read_csv('./Datasets/Data_Combinada.csv')
C:\Users\Usuario\AppData\Local\Temp\ipykernel_11096\2914296293.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  combined_data.filter(items=[ 'sexo','positiva_droga', 'positiva_alcohol', 'tipo_vehiculo', 'tipo_persona', 'rango_edad', 'tipo_accidente'])[combined_data['tipo_persona'] == 'Conductor'][combined_data['tipo_vehiculo'].isnull()][combined_data['sexo'] == 'Desconocido']


,sexo,positiva_droga,positiva_alcohol,tipo_vehiculo,tipo_persona,rango_edad,tipo_accidente
275,Desconocido,0.0,NaN,NaN,Conductor,Desconocido,Colisión lateral
321,Desconocido,0.0,0.0,NaN,Conductor,Desconocido,Colisión lateral
601,Desconocido,0.0,NaN,NaN,Conductor,Desconocido,Colisión lateral
626,Desconocido,0.0,0.0,NaN,Conductor,Desconocido,Colisión múltiple
720,Desconocido,0.0,0.0,NaN,Conductor,Desconocido,Colisión múltiple
...,...,...,...,...,...,...,...
74077,Desconocido,0.0,0.0,NaN,Conductor,Desconocido,Atropello a persona
74094,Desconocido,0.0,0.0,NaN,Conductor,Desconocido,Atropello a persona
74312,Desconocido,0.0,NaN,NaN,Conductor,Desconocido,Atropello a persona
74321,Desconocido,0.0,0.0,NaN,Conductor,Desconocido,Caída


Visto el dataframe anterior observamos que nuestra teoría de que estas entradas podrían representar a personas que huyeron de la escena del accidente tiene sentido por:

1. **Falta de información detallada**: La ausencia de detalles como el tipo de vehículo y el sexo del conductor sugiere que no hubo una interacción exhaustiva con estas personas después del accidente. La presencia de resultados negativos en las pruebas de drogas y alcohol, en medio de otros datos desconocidos, puede indicar errores humanos, ya sea por parte del oficial que atendió el accidente o de la persona que recopiló y registró los datos en el CSV.

2. **Resultados negativos en pruebas, pero falta de otros detalles**: Si bien algunos conductores dieron negativo en las pruebas de drogas y alcohol, la falta de información complementaria es llamativa. Es raro que un conductor que cooperó lo suficiente como para someterse a una prueba luego no proporcionara otros detalles esenciales. Esto puede sugerir un error en la recolección o registro de la información, o posiblemente que el conductor huyó después de la prueba.

3. **Información sobre el tipo de accidente**: Aunque falta información detallada sobre el conductor, sí hay datos sobre el tipo de accidente. Esto podría indicar que, aunque el conductor en cuestión no proporcionó detalles o se fugó, otros involucrados en el accidente o testigos pudieron haber dado testimonio sobre la naturaleza del choque, lo que permitió clasificar el tipo de accidente.

Una vez analizado lo anterior añadimos una columna para los conductores fugados

In [4]:
# Crear la columna 'fugado' con valores por defecto en 'No'
combined_data['fugado'] = 'No'

# Identificar las filas que cumplen con las condiciones que sugieren un posible fugado y asignarles 'Sí'
condition = (combined_data['tipo_persona'] == 'Conductor') & (combined_data['tipo_vehiculo'].isnull()) & (combined_data['sexo'] == 'Desconocido')
combined_data.loc[condition, 'fugado'] = 'Si'

combined_data['fugado'].value_counts()

No    74576
Si      385
Name: fugado, dtype: int64

Cambiamos el 'tipo_vehiculo' de los peatones a 'No' para eliminar datos nulos

In [5]:
combined_data['tipo_persona'].replace({'Peatón': 'No'}, inplace=True)

Verificamos si siguen existiendo nulos en 'tipo_vehiculo' que no sean fugados ni peatones

In [6]:
filas_con_nulos = combined_data['tipo_vehiculo'].isnull()
combined_data[filas_con_nulos][combined_data['fugado'] == 'No'][combined_data['tipo_persona'] != 'No']

C:\Users\Usuario\AppData\Local\Temp\ipykernel_11096\1340320582.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  combined_data[filas_con_nulos][combined_data['fugado'] == 'No'][combined_data['tipo_persona'] != 'No']


,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga,fugado
5391,2022S003477,11/02/2022,19:25:00,CALL. PEDRO RICO / CALL. ARZOBISPO MORCILLO,25,8.0,FUENCARRAL-EL PARDO,Colisión fronto-lateral,Despejado,NaN,Conductor,De 30 a 34 años,Mujer,NaN,NaN,"441574,126","4481237,895",0.0,0.0,No
44829,2022S041133,30/12/2022,9:15:00,"CALL. CAVANILLES, 46",46,3.0,RETIRO,Colisión lateral,Despejado,NaN,Conductor,De 50 a 54 años,Hombre,NaN,NaN,"442968,147","4472839,466",0.0,0.0,No
44830,2022S041133,30/12/2022,9:15:00,"CALL. CAVANILLES, 46",46,3.0,RETIRO,Colisión lateral,Despejado,NaN,Conductor,De 55 a 59 años,Mujer,NaN,NaN,"442968,147","4472839,466",0.0,0.0,No
56861,2023S012840,01/04/2023,8:00:00,"AUTOV. M-30 KM 22,700 CALZADA 1",22NC70,9.0,MONCLOA-ARAVACA,Colisión lateral,Despejado,NaN,Conductor,De 50 a 54 años,Hombre,14.0,Sin asistencia sanitaria,437049.594,4478099.656,0.0,0.0,No
60411,2023S017628,28/04/2023,14:07:00,"CALL. DOCTOR TOLOSA LATOUR, 16",16,12.0,USERA,Alcance,NaN,NaN,Conductor,De 50 a 54 años,Hombre,NaN,NaN,440392.542,4469516.026,0.0,0.0,No
60423,2023S017637,28/04/2023,16:35:00,CARRETERA VILLAVERDE VALLECAS,0,13.0,PUENTE DE VALLECAS,Colisión fronto-lateral,NaN,NaN,Conductor,De 50 a 54 años,Hombre,NaN,NaN,443682.366,4468805.152,0.0,0.0,No
64985,2023S021600,02/06/2023,11:35:00,CALL. MENDEZ ALVARO / METRO. AVENIDA DE LA PAZ,97,13.0,PUENTE DE VALLECAS,Colisión fronto-lateral,Despejado,NaN,Conductor,Desconocido,Hombre,NaN,NaN,442514.278,4471555.855,0.0,0.0,No


### Reemplazar datos nulos de 'tipo_vehiculo'

Reemplazamos los datos de las personas con 'tipo_vehiculo' null dependiendo de la frecuencia del tipo de vehículo y el sexo, excluyendo los que sean sexo 'Desconocido', ya que como corroboramos arriba son los fugados

In [7]:
# Primero, calculamos la frecuencia de 'tipo_vehiculo' para cada 'sexo', excluyendo 'sexo' == 'Desconocido'
vehicle_type_freq = combined_data[combined_data['sexo'] != 'Desconocido'].groupby('sexo')['tipo_vehiculo'].value_counts(normalize=True).unstack().fillna(0)

# A continuación, para cada sexo, seleccionamos el tipo de vehículo más frecuente
most_frequent_vehicle = vehicle_type_freq.idxmax(axis=1)

# Rellenamos los datos nulos de 'tipo_vehiculo' basándonos en la frecuencia del tipo de vehículo para cada sexo
for sex, vehicle in most_frequent_vehicle.items():
    condition = (combined_data['sexo'] == sex) & (combined_data['tipo_vehiculo'].isnull())
    combined_data.loc[condition, 'tipo_vehiculo'] = vehicle

# Verificamos si aún hay valores nulos en 'tipo_vehiculo'
combined_data['tipo_vehiculo'].isnull().sum()


385

Observamos que la cantidad de nulos actuales en 'tipo_vehiculo' coincide con la cantidad de fugados

## Predecimos los sexos desconocidos

In [27]:
count_sexo = combined_data['sexo'].value_counts()
total_desconocidos = count_sexo['Desconocido']
total_desconocidos

6337

In [33]:
nulls_in_sexo = combined_data['sexo'].isnull().sum()

print(f"Valores nulos en 'sexo' : {nulls_in_sexo}")

nulls_in_alcohol = combined_data['positiva_alcohol'].isnull().sum()

print(f"Valores nulos en 'alcohol' : {nulls_in_alcohol}")

Valores nulos en 'sexo' : 0
Valores nulos en 'alcohol' : 341


Cambiamos los nulos de 'positiva_alcohol' por 0, para poder hacer la predicción del sexo con mayor precisión

In [34]:
combined_data['positiva_alcohol'] = combined_data['positiva_alcohol'].fillna(0)

In [35]:
# Crear tabla de contingencia
contingencia_alcohol = pd.crosstab(combined_data['sexo'], combined_data['positiva_alcohol'], dropna=False)
contingencia_alcohol

positiva_alcohol,0.0,1.0
sexo,,
Desconocido,6337,0
Hombre,44467,2035
Mujer,21787,335


Calculamos el total de registros que dieron positivo en la prueba de alcoholemia, excluyendo a los "Desconocidos". Determinamos la proporción de hombres y mujeres que dieron positivo en la prueba. Basándonos en estas proporciones, distribuimos los registros "Desconocido" para inferir cuántos de ellos serían hombres y cuántas mujeres.

In [36]:
# Calcular la proporción de hombres y mujeres que dieron positivo
total_positiva_alcohol = contingencia_alcohol[1].sum()
proporcion_hombres_positiva_alcohol = contingencia_alcohol.loc['Hombre', 1] / total_positiva_alcohol
proporcion_mujeres_positiva_alcohol = 1 - proporcion_hombres_positiva_alcohol

# Distribuir los registros "Desconocido"
registros_desconocidos = contingencia_alcohol.loc['Desconocido', 0] # Solo considero 0 porque 1 para 'Desconocido' es 0
hombres_inferidos = round(registros_desconocidos * proporcion_hombres_positiva_alcohol)
mujeres_inferidas = registros_desconocidos - hombres_inferidos

(hombres_inferidos, mujeres_inferidas)


(5441, 896)

Aproximando que 5441 de los desconocidos son hombres y 896 mujeres; sustituimos los valores de los desconocidos

In [37]:
# Sustituimos los valores desconocidos de manera aleatoria
nuevos_valores_sexo = ['Hombre'] * hombres_inferidos + ['Mujer'] * mujeres_inferidas
np.random.shuffle(nuevos_valores_sexo)
indices_desconocidos = combined_data[combined_data['sexo'] == 'Desconocido'].index
combined_data.loc[indices_desconocidos, 'sexo'] = nuevos_valores_sexo

# Verificar el cambio en la distribución de la columna 'sexo'
combined_data['sexo'].value_counts()

Hombre    51943
Mujer     23018
Name: sexo, dtype: int64

Cambiamos 'Hombre' y 'Mujer' por M y H respectivamente para facilitar la escritura y la velocidad de procesamiento

In [38]:
combined_data['sexo'].replace({'Hombre': 'H', 'Mujer': 'M'}, inplace=True)

# Verificar los cambios
combined_data['sexo'].value_counts()

H    51943
M    23018
Name: sexo, dtype: int64

In [39]:
combined_data.to_csv('./Datasets/Data_Combinada.csv', index=False)